In [3]:
# All the variables are imported from LP.py file
import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side, numbers
import os
import sys
import warnings
import re
import time
import shutil
from openpyxl import utils
warnings.simplefilter("ignore", category=UserWarning, lineno=329, append=False)
warnings.filterwarnings('ignore', message='The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.*',
                       category=FutureWarning)
pd.set_option('future.no_silent_downcasting', True)

# Go up one level from current directory
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from LP import *

print('Calculating compensation elements and running checks on data\nProperty of Lens Consulting & Lens Digital\nRelease version: 1.0\n\n')

# Creating a list for files with issues
unprocessed_files = {}

# Initialize ultimate df
ultimate_df = pd.DataFrame()


fact_sti_threshold = 0.05

# List to drop columns that are not required for Rus version
expected_columns = [company_name, dep_level_1, dep_level_2, dep_level_3, dep_level_4, dep_level_5, dep_level_6,
                             job_title, employee_code, manager_code, man_emp, performance, n_level, expat, gender_id, bod,
                             hired_date, tenure, region_client_fill, region, internal_grade, grade, function_code,
                             subfunction_code, specialization_code, function, subfunction, specialization, salary_rate,
                             monthly_salary, number_monthly_salaries, additional_pay, sti_eligibility, fact_sti,
                             target_sti, lti_eligibility, fact_lti, target_lti_per, lti_prog_1, fact_lti_1,
                             target_lti_1, lti_pay_freq_1, lti_prog_2, fact_lti_2, target_lti_2, lti_pay_freq_2,
                             lti_prog_3, fact_lti_3, target_lti_3, lti_pay_freq_3, comments]

# List to drop columns that are not required for ENG version
expected_columns_eng = [company_name_eng, dep_level_1_eng, dep_level_2_eng, dep_level_3_eng, dep_level_4_eng, 
                                 dep_level_5_eng, dep_level_6_eng, job_title_eng, employee_code_eng, manager_code_eng, 
                                 man_emp_eng, performance_eng, n_level_eng, expat_eng, gender_id_eng, bod_eng, hired_date_eng, 
                                 tenure_eng, region_client_fill_eng, region_eng, internal_grade_eng, grade_eng, function_code_eng, 
                                 subfunction_code_eng, specialization_code_eng, function_eng, subfunction_eng, specialization_eng, 
                                 salary_rate_eng, monthly_salary_eng, number_monthly_salaries_eng, additional_pay_eng, 
                                 sti_eligibility_eng, fact_sti_eng, target_sti_eng, lti_eligibility_eng, fact_lti_eng, 
                                 target_lti_per_eng, lti_prog_1_eng, fact_lti_1_eng, target_lti_1_eng, lti_pay_freq_1_eng, 
                                 lti_prog_2_eng, fact_lti_2_eng, target_lti_2_eng, lti_pay_freq_2_eng, lti_prog_3_eng, 
                                 fact_lti_3_eng, target_lti_3_eng, lti_pay_freq_3_eng, comments_eng]


# Additional columns from General Info sheet from the SDFs
additional_cols = [gi_sector, gi_origin, gi_headcount_cat, gi_revenue_cat, gi_contact_name, 
                   gi_title, gi_tel, gi_email, 'SDF Language']


# Setting the columns in the final df
final_cols = expected_columns + additional_cols

# Creating the final df
ultimate_df = pd.DataFrame(columns=final_cols)


# Function to enter the file paths
def get_valid_path(prompt):
    while True:
        path = input(prompt)
        path = path.replace("\\", "/")  # Replace backslashes with forward slashes
        if os.path.isdir(path):
            return path
        else:
            print("Invalid path. Please try again.")


input_folder = get_valid_path("Please enter the path to the input folder: ")
output_folder = get_valid_path("Please enter the path to the output folder: ")
print(f'\n')

Calculating compensation elements and running checks on data
Property of Lens Consulting & Lens Digital
Release version: 1.0






### Загрузка файла

In [2]:
# # Function to validate sheet names in Excel file
# def validate_excel_sheets(file_path):
#     """
#     Check if Excel file has required sheet combinations.
#     Returns: tuple (is_valid, file_type, missing_sheets)
#     - is_valid: True if file has valid sheet combination
#     - file_type: 'english', 'russian', or None
#     - missing_sheets: list of missing required sheets
#     """
#     try:
#         # Load workbook to get sheet names
#         wb = load_workbook(file_path, read_only=True)
#         sheet_names = wb.sheetnames
#         wb.close()
        
#         # Define required sheet combinations
#         english_sheets = {'General Information', 'Salary Data'}
#         russian_sheets = {'Общая информация', 'Данные'}
        
#         # Check if file has English sheets
#         if english_sheets.issubset(set(sheet_names)):
#             return True, 'english', []
        
#         # Check if file has Russian sheets
#         elif russian_sheets.issubset(set(sheet_names)):
#             return True, 'russian', []
        
#         # File doesn't have required sheets
#         else:
#             # Determine what's missing
#             missing_eng = list(english_sheets - set(sheet_names))
#             missing_rus = list(russian_sheets - set(sheet_names))
            
#             # Return the combination that's closer to being complete
#             if len(missing_eng) <= len(missing_rus):
#                 return False, 'english', missing_eng
#             else:
#                 return False, 'russian', missing_rus
                
#     except Exception as e:
#         return False, None, [f"Error reading file: {str(e)}"]

# # PRE-VALIDATION: Check all files before processing
# print("Step 1: Validating file structure...")
# validation_start = time.time()

# valid_files = []
# invalid_sheet_files = {}
# file_types = {}

# all_excel_files = [f for f in os.listdir(input_folder) 
#                    if f.endswith(('.xlsx', '.xls', '.xlsm'))]

# print(f"Found {len(all_excel_files)} Excel files to validate...")

# for i, file in enumerate(all_excel_files, 1):
#     file_path = os.path.join(input_folder, file)
    
#     # Show progress every 50 files
#     if i % 50 == 0 or i == len(all_excel_files):
#         print(f"Validating... {i}/{len(all_excel_files)}")
    
#     is_valid, file_type, missing_sheets = validate_excel_sheets(file_path)
    
#     if is_valid:
#         valid_files.append(file)
#         file_types[file] = file_type
#     else:
#         invalid_sheet_files[file] = missing_sheets

# validation_end = time.time()

# # Report validation results
# print(f"\nValidation completed in {validation_end - validation_start:.2f} seconds")
# print(f"Valid files: {len(valid_files)}")
# print(f"Invalid files: {len(invalid_sheet_files)}")

# # Handle different scenarios
# if len(valid_files) == 0:
#     print("\nNo valid files found! Please check your files and try again.")
#     exit()

# elif len(invalid_sheet_files) > 0:
#     # Some files have issues - ask user for confirmation
#     print(f"\nFound {len(invalid_sheet_files)} files with missing required sheets:")
#     for file, missing_sheets in invalid_sheet_files.items():
#         print(f"  {file}: Missing {missing_sheets}")
    
#     print(f"\n{len(valid_files)} files are valid and ready for processing.")
#     user_input = input(f"Do you want to continue processing the {len(valid_files)} valid files? (y/n): ").lower().strip()
    
#     if user_input != 'y':
#         print("Processing cancelled.")
#         exit()
    
#     print(f"Proceeding with processing {len(valid_files)} valid files...\n")

# else:
#     # All files are valid - continue automatically
#     print(f"All {len(valid_files)} files passed validation. Proceeding with processing...\n")

In [4]:
start_time = time.time()


# Seeting the counter for the files
counter = 0


# Iterate through all the files in the input folder
process_start = time.time()

for file in os.listdir(input_folder):
    # Check if the file is an Excel file
    if file.endswith('.xlsx') or file.endswith('.xls') or file.endswith('.xlsm'):

        counter += 1

        try:
            print(f"Processing file {counter}: {file}")
            # Process the Excel file
            file_path = os.path.join(input_folder, file)

            if 'Salary Data' in pd.ExcelFile(file_path).sheet_names:

                #Processing the english files
                # Exporting the dataframe from an excel file
                
                # For SDFs
                df = pd.read_excel(file_path, sheet_name=rem_data_eng, header=6)

               
                # Apply cleaning to column names
                df.columns = [re.sub(r'\s+', ' ', str(col).replace('\n', ' ').replace('\r', ' ')).strip() 
                              for col in df.columns]


                # Check if all expected columns are present
                missing_columns_rem_data = [col for col in expected_columns_eng if col not in df.columns]
    
                if missing_columns_rem_data:
                    # If any columns are missing, skip this file
                    unprocessed_files[os.path.basename(file_path)] = missing_columns_rem_data
                    continue
                
                # leaving only required columns
                df = df[expected_columns_eng]

                # print(df.dtypes)
                df_company = pd.read_excel(file_path, sheet_name=company_data_eng, header=1)
                df_company = df_company.iloc[:, 2:]
    
                # Taking the data from the General Info sheet
                extract_company_name = df_company.iloc[0, 1]
                extract_sector = df_company.iloc[1, 1]
                extract_origin = df_company.iloc[2, 1]
                extract_heacount_cat = df_company.iloc[3, 1]
                extract_revenue_cat = df_company.iloc[4, 1]
                extract_contact_name = df_company.iloc[5, 1]
                extract_title = df_company.iloc[6, 1]
                extract_tel = df_company.iloc[7, 1]
                extract_email = df_company.iloc[8, 1]

                # Setting columns names to the russian version
                df.columns = expected_columns

                # Adding the information from the General Info to the temp df
                df[gi_sector] = extract_sector
                df[gi_origin] = extract_origin
                df[gi_headcount_cat] = extract_heacount_cat
                df[gi_revenue_cat] = extract_revenue_cat
                df[gi_contact_name] = extract_contact_name
                df[gi_title] = extract_title
                df[gi_tel] = extract_tel
                df[gi_email] = extract_email
                df['SDF Language'] = 'ENG'


                # Defining columns where there are empty cells (blanks) that needs to be dropped from df
                rows_to_drop = [company_name, job_title]
    
                # Cleaning all the blanks from the columns
                for column in rows_to_drop:
                    df[column] = df[column].replace('', np.nan)
    
                # Dropping rows where company name and title are empty at the same time
                df.dropna(subset=[company_name, job_title], how = 'all', inplace=True)

                # Filling in the company name
                df[company_name] = extract_company_name
                df[gi_company_name] = extract_company_name
               
                # Save the processed DataFrame to the output folder
                ultimate_df = pd.concat([ultimate_df, df])

            else:
                #Processing the russian files
                # Exporting the dataframe from an excel file

                # #For SDFs
                # df = pd.read_excel(file_path, sheet_name=rem_data, header=6)

                #For Combined
                df = pd.read_excel(file_path, sheet_name=rem_data, header=6)
                # print('--------------')
                # print(df.columns)
                # print('--------------')
                
                # Apply cleaning to column names
                df.columns = [re.sub(r'\s+', ' ', str(col).replace('\n', ' ').replace('\r', ' ')).strip() 
                              for col in df.columns]

                # print('--------------')
                # print(df.columns)
                # print('--------------')
                # print(expected_columns_eng)

                # Check if all expected columns are present
                missing_columns_rem_data = [col for col in expected_columns if col not in df.columns]
    
                if missing_columns_rem_data:
                    # If any columns are missing, skip this file
                    unprocessed_files[os.path.basename(file_path)] = missing_columns_rem_data
                    continue
                
                # leaving only required columns
                df = df[expected_columns]

                # print(df.dtypes)
                df_company = pd.read_excel(file_path, sheet_name=company_data, header=1)
                df_company = df_company.iloc[:, 2:]
    
                # Taking the data from the General Info sheet
                extract_company_name = df_company.iloc[0, 1]
                extract_sector = df_company.iloc[1, 1]
                extract_origin = df_company.iloc[2, 1]
                extract_heacount_cat = df_company.iloc[3, 1]
                extract_revenue_cat = df_company.iloc[4, 1]
                extract_contact_name = df_company.iloc[5, 1]
                extract_title = df_company.iloc[6, 1]
                extract_tel = df_company.iloc[7, 1]
                extract_email = df_company.iloc[8, 1]

                # Adding information from the company information tab
                df[gi_company_name] = extract_company_name
                df[gi_sector] = extract_sector
                df[gi_origin] = extract_origin
                df[gi_headcount_cat] = extract_heacount_cat
                df[gi_revenue_cat] = extract_revenue_cat
                df[gi_contact_name] = extract_contact_name
                df[gi_title] = extract_title
                df[gi_tel] = extract_tel
                df[gi_email] = extract_email
                df['SDF Language'] = 'RUS'

                # Defining columns where there are empty cells (blanks) that needs to be dropped from df
                rows_to_drop = [company_name, job_title]
    
                # Cleaning all the blanks from the columns
                for column in rows_to_drop:
                    df[column] = df[column].replace('', np.nan)
    
                # Dropping rows where company name and title are empty at the same time
                df.dropna(subset=[company_name, job_title], how = 'all', inplace=True)

                # Filling in the company name
                df[company_name] = extract_company_name
                df[gi_company_name] = extract_company_name

                # Save the processed DataFrame to the output folder
                ultimate_df = pd.concat([ultimate_df, df])
        
        except Exception as e:
            unprocessed_files[os.path.basename(file_path)] = str(e)


proces_end = time.time()
print(f'Files processed in: {proces_end - process_start}')

if len(unprocessed_files) == 0:
    print(f"\nAll files were processed and concated")
else:
    for file, issue in unprocessed_files.items():
        print(f'\n')
        print("=" * 20 + " WARNING! " + "=" * 20)
        print(f"List of unprocessed files:")
        print(f"File: {file}, Issue: {issue}")
        
    # Create unprocessed folder if it doesn't exist
    # Create unprocessed folder if it doesn't exist
unprocessed_folder = os.path.join(input_folder, 'unprocessed')
os.makedirs(unprocessed_folder, exist_ok=True)

# Copy unprocessed files to the unprocessed folder (overwrite if exists)
if unprocessed_files:
    print(f"\nCopying {len(unprocessed_files)} unprocessed files to 'unprocessed' folder...")

    for file_name in unprocessed_files.keys():
        source_path = os.path.join(input_folder, file_name)
        destination_path = os.path.join(unprocessed_folder, file_name)

        try:
            if os.path.exists(source_path):
                # Если файл уже есть в папке unprocessed — удалим его
                if os.path.exists(destination_path):
                    os.remove(destination_path)
                shutil.copy2(source_path, destination_path)
                print(f"Copied: {file_name}")
        except Exception as e:
            print(f"Failed to copy {file_name}: {str(e)}")

Processing file 1: 12_Storeez_Salary_data_ru_2025.xlsx


c:\Users\s_ivc\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\s_ivc\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\s_ivc\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Processing file 2: 3Logic_Group_Salary data_rus_2025.xlsx
Files processed in: 1.9184560775756836

All files were processed and concated


c:\Users\s_ivc\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\s_ivc\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


### Первичная обработка

- Числовые поля очищены от запятых/пробелов и приведены к float.
- Категориальные поля приведены к строке.
- Значения Yes/No и пол унифицированы.
- Неправильные/лишние значения заменены на NaN.
- STI/LTI обнулены для сотрудников без права на них.

In [5]:
# Preprocess the data

# Defining columns where ',' will be replaced with '.' so that it is recognized as a number
columns_to_convert = [monthly_salary, salary_rate, number_monthly_salaries, fact_sti, fact_lti,
                      target_sti, target_lti_per, additional_pay]


# Replacing ',' with '.' and turning back to floats
for column in columns_to_convert:
    ultimate_df[column] = ultimate_df[column].astype(str).str.replace(',', '.').str.replace(u'\xa0', '')
    ultimate_df[column] = pd.to_numeric(ultimate_df[column], errors='coerce')



# Choose columns to convert to str
str_columns_to_norm = [gender_id, sti_eligibility, lti_eligibility, expat]

# Convert columns to str
for column in str_columns_to_norm:
    ultimate_df[column] = ultimate_df[column].astype(str)


# Function the normalize values 'Yes' and 'No' values
def normalize_eligibility(value):
    if isinstance(value, str):
        value = value.strip().lower()

        if value in ['да', 'д', 'yes', 'y']:
            return "Да"
        else:
            return "Нет"

    elif pd.isnull(value):
        return "Нет"

    return value
    
#removes 'nan' values after they were stringed
ultimate_df[sti_eligibility] = ultimate_df[sti_eligibility].replace('nan', np.nan) 
ultimate_df[lti_eligibility] = ultimate_df[lti_eligibility].replace('nan', np.nan)
ultimate_df[expat] = ultimate_df[expat].replace('nan', np.nan)

# Normalizing 'Yes' and 'No' values 
ultimate_df[sti_eligibility] = ultimate_df[sti_eligibility].apply(normalize_eligibility)
ultimate_df[lti_eligibility] = ultimate_df[lti_eligibility].apply(normalize_eligibility)
ultimate_df[expat] = ultimate_df[expat].apply(normalize_eligibility)


# Function to set np.nan values for variable payments
def adjust_target(df, sti_eligibility, target_sti):
    df.loc[df[sti_eligibility] == 'Нет', target_sti] = np.nan
    return df


#Setting to np.nan for STI and LTI values if the empployee is not eligible
ultimate_df = adjust_target(ultimate_df, sti_eligibility, target_sti)
ultimate_df = adjust_target(ultimate_df, lti_eligibility, target_lti_per)


#Function to normalize gender
def normalize_gender(value):
    if isinstance(value, str):
        value = value.strip().lower()

        if value in ["male", "мужской", "муж", "m", "м-й", 'м', 'мужчина']:
            return "М"
        elif value in ["female", "женский", "жен", "f", "ж-й", 'ж', 'женщина']:
            return "Ж"
    elif np.isnan(value):
        return np.nan

    return value

# Normalizing the gender values
ultimate_df[gender_id] = ultimate_df[gender_id].replace('nan', np.nan)
ultimate_df[gender_id] = ultimate_df[gender_id].apply(normalize_gender)

In [6]:
# Function to lowercase
def lowercase_column(df, column_name):
    """
    Convert a specified column to lowercase strings in place.
    
    Parameters:
    df (pd.DataFrame): The input dataframe (modified in place)
    column_name (str): Name of the column to convert
    
    Returns:
    None
    """
    df[column_name] = df[column_name].astype(str).str.lower()


lowercase_column(ultimate_df, region)

- Замена значений в колонках по словарям (с сохранением старых, если нет совпадения) - автоматическая замена столбцов
- Уточнение названий регионов и привязка их к макрорегионам - автоматическая вставка столбцов

Если колонка не найдена, выводится предупреждение.

In [7]:
# Function to assign values based on a mapping
def translate_values(df, columns, translation_map):
    """
    Translate values in specified DataFrame column(s) using a provided mapping dictionary.
    
    Parameters:
    df: pandas DataFrame
    columns: str or list of str, column name(s) to translate
    translation_map: dict, mapping of original values to translated values
    
    Returns:
    pandas DataFrame with translated values
    """
    # Create a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Ensure columns is a list for uniform processing
    if isinstance(columns, str):
        columns = [columns]
    
    # Apply translation to each specified column
    for col in columns:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].map(translation_map).fillna(df_copy[col])
        else:
            print(f"Warning: Column '{col}' not found in DataFrame")
    
    return df_copy

In [8]:
def map_column_values(df, check_column, amend_column, mapping_dict):
    """
    Check values in one column and assign mapped values to another column.
    
    Parameters:
    df: pandas DataFrame
    check_column: str, name of the column to check values in
    amend_column: str, name of the column to assign mapped values to
    mapping_dict: dict, mapping of check_column values to amend_column values
    
    Returns:
    pandas DataFrame with amended values
    """
    # Create a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Check if both columns exist
    if check_column not in df_copy.columns:
        print(f"Warning: Check column '{check_column}' not found in DataFrame")
        return df_copy
    
    if amend_column not in df_copy.columns:
        print(f"Warning: Amend column '{amend_column}' not found in DataFrame")
        return df_copy
    
    # Map values from check_column to amend_column
    df_copy[amend_column] = df_copy[check_column].map(mapping_dict).fillna(df_copy[amend_column])
    
    return df_copy

In [9]:
# Apply translations using the tranlsation function | Converting English version to Russian
ultimate_df = translate_values(ultimate_df, [expat, sti_eligibility, lti_eligibility], yes_no_map)
ultimate_df = translate_values(ultimate_df, man_emp, manager_spec_map)
ultimate_df = translate_values(ultimate_df, performance, performance_map)
ultimate_df = translate_values(ultimate_df, gender_id, gender_map)
ultimate_df = translate_values(ultimate_df, region, region_match_map)
ultimate_df = translate_values(ultimate_df, tenure, tenure_map)
ultimate_df = translate_values(ultimate_df, [lti_prog_1, lti_prog_2, lti_prog_3], lti_map)
ultimate_df = translate_values(ultimate_df, gi_sector, sector_map)
ultimate_df = translate_values(ultimate_df, gi_origin, origin_map)
ultimate_df = translate_values(ultimate_df, gi_revenue_cat, revenue_map)

# Assigning proper russian region names
ultimate_df = translate_values(ultimate_df, region, final_region)

# Assigning macroregion names
ultimate_df[macroregion] = np.nan
ultimate_df = map_column_values(ultimate_df, region, macroregion, region_to_macroregion_map)

# Assigning russian names of Functions, Subfunction and Specializations based on codes
ultimate_df = map_column_values(ultimate_df, function_code, function, function_map)
ultimate_df = map_column_values(ultimate_df, subfunction_code, subfunction, subfunction_map)
ultimate_df = map_column_values(ultimate_df, specialization_code, specialization, specialization_map)

In [10]:
# Running the required checks
check_start = time.time()
print('\nRunning required checks')


Running required checks


### Логические проверки

Грейд — проверка диапазона, целочисленности.

Коды (Fun/Sub/Spec) — проверка, что существуют и есть в словарях.

Регион — проверка на соответствие допустимым значениям.

Иерархия кодов — Subfunction соответствует Function, Specialization соответствует Subfunction.

Зарплатные параметры — ставка и количество окладов в допустимых диапазонах.

Пропуски — проверка, указаны ли оклад/eligibility.

Стаж — проверка корректности значения.

In [11]:
# Ensuring grade value is not None, is in 7 to 30 range and is an integer
def check_grade_validity(df, grade_col, new_col='Grade_Check'):
    """
    Vectorized check if grade values meet all validity criteria:
    1. Not null/empty
    2. Between 7 and 30 (inclusive)
    3. Is an integer
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    grade_col (str): The name of the grade column to check.
    new_col (str): The name of the new column for boolean values.
    
    Returns:
    pd.DataFrame: Dataframe with the new boolean column.
    """
    # Convert to numeric, coercing errors to NaN
    grade_numeric = pd.to_numeric(df[grade_col], errors='coerce')
    
    # Check 1: Not null/empty (after numeric conversion)
    not_null = grade_numeric.notna()
    
    # Check 2: Between 7 and 30 (inclusive)
    in_range = grade_numeric.between(7, 30, inclusive='both')
    
    # Check 3: Is integer (no decimal part)
    is_integer = (grade_numeric == grade_numeric.astype('Int64', errors='ignore'))
    
    # Combine all conditions using vectorized operations
    df[new_col] = not_null & in_range & is_integer
    
    return df

# Usage:
ultimate_df = check_grade_validity(ultimate_df, grade, 'Grade_Check')

In [12]:
# Ensuring that Fun, Sub and Spec code are not empty and are the valide codes from function model
def check_value_in_dict_keys(df, input_column, output_column, valid_dict, dict_name=None):
    """
    Vectorized check if values in input column are valid after cleaning:
    1. Clean the input column (lowercase, remove spaces, uppercase)
    2. Check if not missing (special case for specialization_map)
    3. Check if value is in the dictionary keys
    
    Parameters:
    df (pd.DataFrame): The input dataframe
    input_column (str): The name of the input column to check
    output_column (str): The name of the output column for boolean results
    valid_dict (dict): Dictionary whose keys are the valid values to check against
    dict_name (str): Name of the dictionary (for special handling of 'spec')
    
    Returns:
    pd.DataFrame: Dataframe with the new output column
    """
    # Step 1: Apply cleaning to the input column directly - vectorized string operations
    df[input_column] = df[input_column].str.lower().str.replace(r'\s+', "", regex=True).str.upper()
    
    # Step 2: Check if not missing - vectorized
    not_missing = df[input_column].notna()
    
    # Step 3: Check if value is in the dictionary keys - vectorized
    in_dict_keys = df[input_column].isin(valid_dict.keys())
    
    # Special case: for specialization_map, empty values are allowed
    if dict_name == 'spec':
        # For specialization_map: True if missing OR (not missing AND in keys)
        df[output_column] = (~not_missing) | (not_missing & in_dict_keys)
    else:
        # Standard case: True only if not missing AND in keys
        df[output_column] = not_missing & in_dict_keys
    
    return df

# Checking the codes validity
ultimate_df = check_value_in_dict_keys(ultimate_df, function_code, 'Fun Code Validity', function_map)
ultimate_df = check_value_in_dict_keys(ultimate_df, subfunction_code, 'Sub Code Validity', subfunction_map)
ultimate_df = check_value_in_dict_keys(ultimate_df, specialization_code, 'Spec Code Validity', specialization_map, 'spec')

# Function to check that regions are valid
def check_value_in_regions(df, input_column, output_column, valid_dict, ):
    """
    Vectorized check if values in input column are valid after cleaning:
    1. Clean the input column (lowercase, remove spaces, uppercase)
    2. Check if not missing (special case for specialization_map)
    3. Check if value is in the dictionary keys
    
    Parameters:
    df (pd.DataFrame): The input dataframe
    input_column (str): The name of the input column to check
    output_column (str): The name of the output column for boolean results
    valid_dict (dict): Dictionary whose keys are the valid values to check against
    dict_name (str): Name of the dictionary (for special handling of 'spec')
    
    Returns:
    pd.DataFrame: Dataframe with the new output column
    """
   
    # Step 2: Check if not missing - vectorized
    not_missing = df[input_column].notna()
    
    # Step 3: Check if value is in the dictionary keys - vectorized
    in_dict_values = df[input_column].isin(set(valid_dict.values()))
    
    # Standard case: True only if not missing AND in keys
    df[output_column] = not_missing & in_dict_values
    
    return df

# Checking the region validity
ultimate_df = check_value_in_regions(ultimate_df, region, 'Region Validity', final_region)

In [13]:
# Fucntion to check that Subfunction belongs to the function
def compare_sub_fun(df, column_1, column_2, num_letters, result_column):
    """
    Сравнивает первые num_letters символов в двух столбцах DataFrame
    и создает новый столбец с булевыми значениями.

    Условия:
    - Если длина значения в column_1 не равна 2, устанавливается False.
    - Если длина значения в column_2 не равна 3, устанавливается False.
    - В остальных случаях сравниваются первые num_letters символов значений column_1 и column_2.

    Параметры:
    df (pd.DataFrame): Исходный DataFrame
    column_1 (str): Название первого столбца
    column_2 (str): Название второго столбца
    num_letters (int): Количество первых символов для сравнения
    result_column (str): Название создаваемого столбца с булевыми значениями

    Возвращает:
    pd.DataFrame: Обновленный DataFrame с новым столбцом
    """

    df_1 = df.copy()

    # Основное сравнение первых num_letters символов
    df_1[result_column] = df_1[column_1].str[:num_letters] == df_1[column_2].str[:num_letters]

    # Если длина column_1 не равна 2 или длина column_2 не равна 3, устанавливаем False
    df_1[result_column] = df_1[result_column] & (df_1[column_1].str.len() == 2) & (df_1[column_2].str.len() == 3)

    return df_1

# Applying the check
ultimate_df = compare_sub_fun(ultimate_df, function_code, subfunction_code, 2, 'Sub in Fun')

In [14]:
# Fucntion to check that Specialization belongs to the subfunction
def compare_spec_sub(df, column_1, column_2, num_letters, result_column):
    """
    Сравнивает первые num_letters символов в двух столбцах DataFrame
    и создает новый столбец с булевыми значениями.
    Условия:
    - Если значение в column_2 пустое (NaN или пустая строка), то автоматически устанавливается True.
    - Если длина значения в column_1 не равна 3, устанавливается False.
    - Если длина значения в column_2 меньше или больше 5 символов (исключая пустые значения), устанавливается False.
    - В остальных случаях сравниваются первые num_letters символов значений column_1 и column_2.
    Параметры:
    df (pd.DataFrame): Исходный DataFrame
    column_1 (str): Название первого столбца
    column_2 (str): Название второго столбца
    num_letters (int): Количество первых символов для сравнения
    result_column (str): Название создаваемого столбца с булевыми значениями
    Возвращает:
    pd.DataFrame: Обновленный DataFrame с новым столбцом
    """
    
    df_1 = df.copy()
    
    # Основное сравнение первых num_letters символов
    df_1[result_column] = df_1[column_1].str[:num_letters] == df_1[column_2].str[:num_letters]
    
    # Если column_2 пустое (NaN или пустая строка), ставим True
    df_1[result_column] = df_1[result_column] | (df_1[column_2].isna() | (df_1[column_2] == ''))
    
    # Если длина column_1 не равна 3, устанавливаем False
    df_1[result_column] = df_1[result_column] & (df_1[column_1].str.len() == 3)
    
    # Если длина column_2 не равна 5 и не пустое значение, устанавливаем False
    df_1[result_column] = df_1[result_column] & ((df_1[column_2].str.len() == 5) | df_1[column_2].isna() | (df_1[column_2] == ''))
    
    return df_1

# Применение функции
ultimate_df = compare_spec_sub(ultimate_df, subfunction_code, specialization_code, 3, 'Spec in Sub')

In [15]:
# Fucntion to check the number of salaries in a year is a valid number
def check_range(df, input_column, output_column, upper, lower):
    """
    Create a boolean column based on range check of input column.
    
    Logic:
    - False: if value is n/a (NaN/None) or outside the specified range
    - True: if value is greater than lower and less than or equal to upper
    
    Parameters:
    df (pd.DataFrame): The input dataframe
    input_column (str): Name of the column to check
    output_column (str): Name of the new column to create with boolean results
    upper (float): Upper bound (inclusive)
    lower (float): Lower bound (exclusive)
    
    Returns:
    None (modifies dataframe in place)
    
    Example:
    check_range_to_boolean(df, 'scores', 'is_valid', upper=1.5, lower=0)
    # True for values in range (0, 1.5], False otherwise
    """
    # Input validation
    if upper <= lower:
        raise ValueError(f"Upper bound ({upper}) must be greater than lower bound ({lower})")
    
    # Convert to numeric, coercing errors to NaN
    numeric_values = pd.to_numeric(df[input_column], errors='coerce')
    
    # Apply the logic using vectorized operations
    # True if: value > lower AND value <= upper
    # False if: value is NaN, <= lower, or > upper
    df[output_column] = (numeric_values > lower) & (numeric_values <= upper)

# Check validity of the salary rate
check_range(ultimate_df, salary_rate, 'Ставка', 1.5, 0)

# Check validity of the number of monthly salaries
check_range(ultimate_df, number_monthly_salaries, 'Число окладов', 13, 11)

In [16]:
# Function to check that whether there are missing values
def no_value(df, col_name, new_col_name):
    df[new_col_name] = ~df[col_name].isna()
    return df

#Checking missing values 
ultimate_df = no_value(ultimate_df, monthly_salary, 'Оклад указан')
ultimate_df = no_value(ultimate_df, sti_eligibility, 'STI_eligibility')
ultimate_df = no_value(ultimate_df, lti_eligibility, 'LTI_eligibility')

#Function to check validity for the employee tenure
def value_check(df, col_name, new_col_name, str):
    df[new_col_name] = (df[col_name].isna()) | (df[col_name] == str)
    return df

ultimate_df = value_check(ultimate_df, tenure, 'Tenure Value', 'Меньше года')

### Requirements by Olga

- Даты к единому виду (год или DD.MM.YYYY).
- Проверки возраста/валидности.
- Валидация грейдов по правилам.
- Уникальные ID сотрудников внутри компании

#### Normalizing DOB to YYYY only

In [17]:
# Function to normalize the DOB to YYYY only
def extract_year_from_dates_vectorized(df, column_name):
    """
    Vectorized function to extract birth year from various date formats in a DataFrame column.
    Only returns years where the person would be at least 18 years old.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    column_name (str): Name of the column containing birth dates
    
    Returns:
    pd.Series: Series with extracted birth years or NaN for invalid/underage dates
    """
    
    current_year = datetime.now().year
    min_birth_year = current_year - 18  # Must be at least 18 years old
    max_birth_year = current_year - 100  # Reasonable maximum age limit
    
    # Start with the column and convert to string, handle NaN
    series = df[column_name].astype(str).str.strip()
    
    # Initialize result series with NaN (using nullable integer dtype)
    result = pd.Series(np.nan, index=series.index, dtype='Int64')
    
    # Handle NaN/None values (they become 'nan' when converted to string)
    nan_mask = (series == 'nan') | (series == '') | series.isna()
    
    # Pattern 1: DD.MM.YYYY, DD/MM/YYYY, DD-MM-YYYY (extract year from position 3)
    pattern1_mask = series.str.match(r'^\d{1,2}[.\-/]\d{1,2}[.\-/]\d{4}$', na=False)
    if pattern1_mask.any():
        years1 = series[pattern1_mask].str.extract(r'^\d{1,2}[.\-/]\d{1,2}[.\-/](\d{4})$')[0]
        years1_numeric = pd.to_numeric(years1, errors='coerce')
        # Check if person is at least 18 years old and create final mask
        valid_years1 = (years1_numeric >= max_birth_year) & (years1_numeric <= min_birth_year) & years1_numeric.notna()
        # Map back to original indices
        final_mask1 = pd.Series(False, index=series.index)
        final_mask1.loc[pattern1_mask] = valid_years1
        if final_mask1.any():
            result.loc[final_mask1] = years1_numeric[valid_years1].values
    
    # Pattern 2: YYYY.MM.DD, YYYY/MM/DD, YYYY-MM-DD (extract year from position 1)
    pattern2_mask = series.str.match(r'^\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2}$', na=False) & ~pattern1_mask
    if pattern2_mask.any():
        years2 = series[pattern2_mask].str.extract(r'^(\d{4})[.\-/]\d{1,2}[.\-/]\d{1,2}$')[0]
        years2_numeric = pd.to_numeric(years2, errors='coerce')
        # Check if person is at least 18 years old and create final mask
        valid_years2 = (years2_numeric >= max_birth_year) & (years2_numeric <= min_birth_year) & years2_numeric.notna()
        # Map back to original indices
        final_mask2 = pd.Series(False, index=series.index)
        final_mask2.loc[pattern2_mask] = valid_years2
        if final_mask2.any():
            result.loc[final_mask2] = years2_numeric[valid_years2].values
    
    # Pattern 3: Just YYYY (4 digits only)
    pattern3_mask = series.str.match(r'^\d{4}$', na=False) & ~pattern1_mask & ~pattern2_mask
    if pattern3_mask.any():
        years3 = pd.to_numeric(series[pattern3_mask], errors='coerce')
        # Check if person is at least 18 years old and create final mask
        valid_years3 = (years3 >= max_birth_year) & (years3 <= min_birth_year) & years3.notna()
        # Map back to original indices
        final_mask3 = pd.Series(False, index=series.index)
        final_mask3.loc[pattern3_mask] = valid_years3
        if final_mask3.any():
            result.loc[final_mask3] = years3[valid_years3].values
    
    # Pattern 4: Fallback - try pandas to_datetime for remaining non-empty values
    remaining_mask = result.isna() & ~nan_mask & ~pattern1_mask & ~pattern2_mask & ~pattern3_mask
    if remaining_mask.any():
        try:
            # Vectorized datetime conversion
            dates_remaining = pd.to_datetime(series[remaining_mask], errors='coerce')
            years4 = dates_remaining.dt.year
            # Check if person is at least 18 years old and create final mask
            valid_years4 = (years4 >= max_birth_year) & (years4 <= min_birth_year) & years4.notna()
            # Map back to original indices
            final_mask4 = pd.Series(False, index=series.index)
            final_mask4.loc[remaining_mask] = valid_years4
            if final_mask4.any():
                result.loc[final_mask4] = years4[valid_years4].values
        except:
            pass  # If vectorized conversion fails, leave as NaN
    
    return result


ultimate_df[bod] = extract_year_from_dates_vectorized(ultimate_df, bod)

C:\Users\s_ivc\AppData\Local\Temp\ipykernel_52176\3745762469.py:71: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates_remaining = pd.to_datetime(series[remaining_mask], errors='coerce')


#### Normalizing dates to DD.MM.YYYY format

In [18]:
# Normalizing the dates to DD.MM.YYYY
def normalize_dates_to_full_vectorized(df, date_column):
    """
    Vectorized function to normalize various date formats to DD.MM.YYYY format.
    Modifies the original DataFrame column in-place.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    date_column (str): Name of the column containing dates
    
    Returns:
    pd.DataFrame: Modified dataframe with normalized dates in DD.MM.YYYY format
    """
    
    # Work with the original column data
    series = df[date_column]
    
    # Initialize result series with NaT (Not a Time)
    result = pd.Series(pd.NaT, index=series.index, dtype='datetime64[ns]')
    
    # Handle already datetime objects
    if pd.api.types.is_datetime64_any_dtype(series):
        result = series.copy()
    else:
        # Convert to string for processing
        series_str = series.astype(str).str.strip()
        
        # Handle NaN/None values
        valid_mask = ~((series_str == 'nan') | (series_str == '') | series.isna())
        
        if valid_mask.any():
            # Pattern 1: Handle 4-digit years (vectorized)
            year_pattern = series_str.str.match(r'^\d{4}$', na=False)
            if year_pattern.any():
                years = pd.to_numeric(series_str[year_pattern], errors='coerce')
                valid_years = (years >= 1900) & (years <= 2100)  # Reasonable year range
                
                # Create proper mask mapping back to original index
                final_year_mask = pd.Series(False, index=series.index)
                final_year_mask.loc[year_pattern] = valid_years
                
                if final_year_mask.any():
                    # Create January 1st dates for valid years - use .values to avoid index alignment
                    year_dates = pd.to_datetime(years[valid_years].astype(str) + '-01-01', errors='coerce')
                    result.loc[final_year_mask] = year_dates.values
            
            # Pattern 2: Handle numeric values (Excel serial dates or timestamps)
            numeric_mask = pd.to_numeric(series_str, errors='coerce').notna() & ~year_pattern
            if numeric_mask.any():
                numeric_values = pd.to_numeric(series_str[numeric_mask], errors='coerce')
                
                # Excel serial dates (typically 1-100000 range)
                excel_mask = (numeric_values >= 1) & (numeric_values <= 100000)
                if excel_mask.any():
                    try:
                        # Excel epoch: 1899-12-30 (correcting Excel's leap year bug)
                        excel_dates = pd.to_datetime('1899-12-30') + pd.to_timedelta(numeric_values[excel_mask], unit='d')
                        
                        # Create proper mask mapping back to original index
                        final_excel_mask = pd.Series(False, index=series.index)
                        final_excel_mask.loc[numeric_mask] = excel_mask
                        
                        if final_excel_mask.any():
                            result.loc[final_excel_mask] = excel_dates.values
                    except:
                        pass
                
                # Unix timestamps (large numbers)
                remaining_numeric = numeric_mask & result.isna()
                if remaining_numeric.any():
                    remaining_values = pd.to_numeric(series_str[remaining_numeric], errors='coerce')
                    timestamp_mask = remaining_values > 100000
                    if timestamp_mask.any():
                        try:
                            unix_dates = pd.to_datetime(remaining_values[timestamp_mask], unit='s', errors='coerce')
                            
                            # Create proper mask mapping back to original index
                            final_timestamp_mask = pd.Series(False, index=series.index)
                            final_timestamp_mask.loc[remaining_numeric] = timestamp_mask
                            
                            if final_timestamp_mask.any():
                                result.loc[final_timestamp_mask] = unix_dates.values
                        except:
                            pass
            
            # Pattern 3: Handle string dates (vectorized with multiple format attempts)
            string_mask = valid_mask & ~year_pattern & ~numeric_mask & result.isna()
            if string_mask.any():
                string_dates = series_str[string_mask]
                
                # Try multiple date formats in order of likelihood
                date_formats = [
                    '%d.%m.%Y',    # DD.MM.YYYY
                    '%d/%m/%Y',    # DD/MM/YYYY  
                    '%d-%m-%Y',    # DD-MM-YYYY
                    '%Y-%m-%d',    # YYYY-MM-DD (ISO format)
                    '%Y/%m/%d',    # YYYY/MM/DD
                    '%Y.%m.%d',    # YYYY.MM.DD
                    '%d.%m.%y',    # DD.MM.YY
                    '%d/%m/%y',    # DD/MM/YY
                    '%d-%m-%y',    # DD-MM-YY
                ]
                
                parsed_dates = pd.Series(pd.NaT, index=string_dates.index)
                
                for date_format in date_formats:
                    remaining_mask = parsed_dates.isna()
                    if remaining_mask.any():
                        try:
                            temp_dates = pd.to_datetime(string_dates[remaining_mask], 
                                                      format=date_format, errors='coerce')
                            parsed_dates.loc[remaining_mask] = temp_dates
                        except:
                            continue
                
                # If no specific format worked, try pandas' flexible parser
                remaining_mask = parsed_dates.isna()
                if remaining_mask.any():
                    try:
                        # Use dayfirst=True for European format preference
                        flexible_dates = pd.to_datetime(string_dates[remaining_mask], 
                                                      dayfirst=True, errors='coerce')
                        parsed_dates.loc[remaining_mask] = flexible_dates
                    except:
                        pass
                
                # Assign parsed dates using .values to avoid index alignment issues
                if parsed_dates.notna().any():
                    valid_parsed = parsed_dates.notna()
                    final_string_mask = pd.Series(False, index=series.index)
                    final_string_mask.loc[string_mask] = valid_parsed
                    result.loc[final_string_mask] = parsed_dates[valid_parsed].values
    
    # Filter out unrealistic dates
    current_year = datetime.now().year
    realistic_mask = (result.dt.year >= 1900) & (result.dt.year <= current_year + 10)
    result = result.where(realistic_mask)
    
    # Format to DD.MM.YYYY string
    formatted_result = result.dt.strftime('%d.%m.%Y')
    
    # Replace 'NaT' strings with actual NaN
    formatted_result = formatted_result.replace('NaT', np.nan)
    
    # Modify the original DataFrame column
    df[date_column] = formatted_result
    
    return df

ultimate_df = normalize_dates_to_full_vectorized(ultimate_df, hired_date) 

In [19]:
def validate_code_grade(df, input_column_1, input_column_2, output_column, validation_dict):
    """
    Vectorized function to validate subfunction codes against grade thresholds.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    input_column_1 (str): Column name containing subfunction codes
    input_column_2 (str): Column name containing grades  
    output_column (str): Column name for output (will be created)
    validation_dict (dict): Dictionary with codes as keys and 'threshold, direction' as values
                           Example: {'ABC': '14, above', 'XYZ': '12, below'}
    
    Returns:
    pd.DataFrame: Modified dataframe with new validation column
    
    Logic:
    - Dictionary rule: 'above' = False if grade > threshold, 'below' = False if grade < threshold
    - Z rule (overrides): codes ending with 'Z' = False if grade < 14
    - Default: NaN for codes not in dict and not ending with Z
    - NaN for missing values
    """
    
    # Get the input columns
    codes = df[input_column_1].copy()
    grades = df[input_column_2].copy()
    
    # Initialize result with NaN
    result = pd.Series(np.nan, index=df.index, dtype='object')
    
    # Handle missing values - return NaN
    missing_mask = codes.isna() | grades.isna()
    
    # Work with valid (non-missing) data
    valid_mask = ~missing_mask
    if not valid_mask.any():
        df[output_column] = result
        return df
    
    valid_codes = codes[valid_mask].astype(str)
    valid_grades = pd.to_numeric(grades[valid_mask], errors='coerce')
    
    # Handle cases where grade conversion failed
    grade_na_mask = valid_grades.isna()
    if grade_na_mask.any():
        # Keep these as NaN in result
        valid_mask_updated = valid_mask.copy()
        valid_mask_updated.loc[valid_mask] = ~grade_na_mask
        valid_mask = valid_mask_updated
        
        if not valid_mask.any():
            df[output_column] = result
            return df
            
        valid_codes = codes[valid_mask].astype(str)
        valid_grades = pd.to_numeric(grades[valid_mask], errors='coerce')
    
    # Initialize valid results with object dtype to handle both bool and NaN
    valid_results = pd.Series(True, index=valid_codes.index, dtype='object')
    
    # RULE 1: Z rule (overrides dictionary) - vectorized
    z_code_mask = valid_codes.str.endswith('Z', na=False)
    if z_code_mask.any():
        z_violation_mask = (valid_grades < 14) & z_code_mask
        valid_results.loc[z_violation_mask] = False
        
        # For Z codes that pass the rule, keep as True
        z_pass_mask = (valid_grades >= 14) & z_code_mask
        valid_results.loc[z_pass_mask] = True
    
    # RULE 2: Dictionary rules (for non-Z codes or codes not processed by Z rule)
    non_z_mask = ~z_code_mask
    
    if non_z_mask.any() and validation_dict:
        # Parse dictionary and create vectorized conditions
        for code, rule_str in validation_dict.items():
            try:
                # Parse rule string: '14, above' -> threshold=14, direction='above'
                parts = [part.strip() for part in rule_str.split(',')]
                if len(parts) != 2:
                    continue
                    
                threshold = float(parts[0])
                direction = parts[1].lower()
                
                # Find matching codes (vectorized)
                code_match_mask = (valid_codes == code) & non_z_mask
                
                if code_match_mask.any():
                    if direction == 'above':
                        # False if grade > threshold (violates 'above' rule)
                        violation_mask = (valid_grades > threshold) & code_match_mask
                        valid_results.loc[violation_mask] = False
                        
                        # True if grade <= threshold (meets 'above' rule)
                        pass_mask = (valid_grades <= threshold) & code_match_mask
                        valid_results.loc[pass_mask] = True
                        
                    elif direction == 'below':
                        # False if grade < threshold (violates 'below' rule)
                        violation_mask = (valid_grades < threshold) & code_match_mask
                        valid_results.loc[violation_mask] = False
                        
                        # True if grade >= threshold (meets 'below' rule)
                        pass_mask = (valid_grades >= threshold) & code_match_mask
                        valid_results.loc[pass_mask] = True
                        
            except (ValueError, IndexError):
                # Skip malformed rules
                continue
        
        # For codes not in dictionary and not ending with Z: set to NaN
        codes_in_dict = pd.Series(False, index=valid_codes.index)
        for code in validation_dict.keys():
            codes_in_dict |= (valid_codes == code)
        
        not_in_dict_not_z = non_z_mask & ~codes_in_dict
        valid_results.loc[not_in_dict_not_z] = np.nan
    
    # For non-Z codes when no dictionary provided: set to NaN
    elif non_z_mask.any():
        valid_results.loc[non_z_mask] = np.nan
    
    # Assign results back to main result series
    result.loc[valid_mask] = valid_results
    
    # Create output column
    df[output_column] = result
    
    return df

ultimate_df = validate_code_grade(
    ultimate_df, 
    subfunction_code,   # input_column_1
    grade,              # input_column_2  
    'Code_Grade_Val',   # output_column
    code_grade_map       # dictionary
)

In [20]:
# Function to generate unique employee IDs within each company
def generate_employee_ids(df, company_column, employee_code_column):
    
    # Create a simple working DataFrame with clean integer index
    work_data = []
    for i in range(len(df)):
        company = df.iloc[i][company_column]
        code = df.iloc[i][employee_code_column]
        # Convert to string, but keep None/NaN as None
        if pd.isna(code):
            code_str = None
        else:
            code_str = str(code)
        work_data.append((i, company, code, code_str))
    
    # Group by company manually
    company_groups = {}
    for orig_idx, company, orig_code, code_str in work_data:
        if pd.notna(company):  # Skip rows with missing company
            if company not in company_groups:
                company_groups[company] = []
            company_groups[company].append((orig_idx, orig_code, code_str))
    
    # Process each company
    new_codes = {}  # orig_idx -> new_code
    
    for company, company_data in company_groups.items():
        # Track codes we've seen and indices needing new IDs
        seen_codes = set()
        indices_needing_ids = []
        
        for orig_idx, orig_code, code_str in company_data:
            if code_str is None:
                # Missing code
                indices_needing_ids.append(orig_idx)
            elif code_str in seen_codes:
                # Duplicate code
                indices_needing_ids.append(orig_idx)
            else:
                # First occurrence
                seen_codes.add(code_str)
                new_codes[orig_idx] = orig_code  # Keep original
        
        # Generate LCR IDs for indices needing them
        if indices_needing_ids:
            for i, orig_idx in enumerate(indices_needing_ids):
                new_id = f'LCR{i+1:03d}'
                new_codes[orig_idx] = new_id
    
    # Update the DataFrame
    result_codes = df[employee_code_column].copy()
    for orig_idx, new_code in new_codes.items():
        result_codes.iloc[orig_idx] = new_code
    
    df[employee_code_column] = result_codes
    return df

ultimate_df = generate_employee_ids(ultimate_df, company_name, employee_code)

Этот кусок кода отвечает за дополнительную проверку права на участие в LTI (Long-Term Incentives) по грейду.

In [21]:
# Check whether it is a high enough grade to be eligible for LTI
def validate_lti_grade(df, input_column, grade_column, output_column):
    """
    Vectorized function to validate grade and response combinations.
    Flags as False when grade < 14 AND input_column = 'Да'.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    input_column (str): Column name containing responses (looking for 'Да')
    grade_column (str): Column name containing grade values
    output_column (str): Column name for output (will be created)
    
    Returns:
    pd.DataFrame: Modified dataframe with new validation column
    
    Logic:
    - False ONLY if: grade < 14 AND input_column exactly equals 'Да'
    - True for all other valid combinations
    - NaN if either grade_column or input_column has missing values
    """
    
    # Get the input columns
    responses = df[input_column].copy()
    grades = df[grade_column].copy()
    
    # Convert grades to numeric, invalid values become NaN
    grades_numeric = pd.to_numeric(grades, errors='coerce')
    
    # Initialize result with True (default for valid cases)
    result = pd.Series(True, index=df.index, dtype='object')
    
    # Handle missing values - set to NaN
    missing_mask = responses.isna() | grades_numeric.isna()
    result.loc[missing_mask] = np.nan
    
    # Work only with non-missing data
    valid_mask = ~missing_mask
    
    if valid_mask.any():
        valid_responses = responses[valid_mask]
        valid_grades = grades_numeric[valid_mask]
        
        # Vectorized condition: grade < 14 AND response exactly equals 'Да'
        violation_mask = (valid_grades < 14) & (valid_responses == 'Да')
        
        # Create final mask for the entire dataframe
        final_violation_mask = pd.Series(False, index=df.index)
        final_violation_mask.loc[valid_mask] = violation_mask
        
        # Set violations to False
        result.loc[final_violation_mask] = False
    
    # Create output column
    df[output_column] = result
    
    return df

ultimate_df = validate_lti_grade(
    ultimate_df, 
    lti_eligibility,        # input_column
    grade,                  # grade_column
    'LTI_Grade_Val'     # output_column
)

#### Calculating the compensation elements

- Полный пересчёт фиксированных и переменных компонентов компенсации.
- Проверки на корректность eligibility (STI/LTI, tenure).
- Проверки на реалистичность (fact_sti_threshold, crazy_ranges).
- Валидации consistency (fact_lti vs sum, fact_lti vs tc, mrot).

Отдельные флаги для сомнительных значений.

In [22]:
# Fucntion to calculate compensation elements
def calculate_compensation_elements(df, 
                                  # Input column names
                                  monthly_salary, salary_rate, number_annual_salaries, additional_pay,
                                  sti_eligibility, tenure, fact_sti, target_sti, fact_lti, target_lti_per,
                                  lti_eligibility,
                                  # Output column names
                                  annual_salary, base_pay, fact_sti_out, fact_sti_out_alt, target_sti_out,
                                  tc_pay, ltip_pay, ltip_pay_alt, ttc_pay, tltip_pay, tdc_pay, ttdc_pay,
                                  # Constants/thresholds
                                  positive_v, negative_v, tenure_value, fact_sti_threshold):
    """
    Vectorized function to calculate various compensation elements.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    
    Input column names (str):
    - monthly_salary: Monthly salary column
    - salary_rate: Salary rate column  
    - number_annual_salaries: Number of annual salaries column
    - additional_pay: Additional pay column
    - sti_eligibility: STI eligibility column
    - tenure: Tenure column
    - fact_sti: Actual STI column
    - target_sti: Target STI column
    - fact_lti: Actual LTI column
    - target_lti_per: Target LTI percentage column
    - lti_eligibility: LTI eligibility column
    
    Output column names (str):
    - annual_salary: Annual salary output column
    - base_pay: Base pay output column
    - fact_sti_out: Filtered actual STI output column
    - fact_sti_out_alt: Alternative actual STI output column
    - target_sti_out: Target STI output column
    - tc_pay: Total cash pay column
    - ltip_pay: LTIP pay column
    - ltip_pay_alt: Alternative LTIP pay column
    - ttc_pay: Target total cash pay column
    - tltip_pay: Target LTIP pay column
    - tdc_pay: Total direct compensation column
    - ttdc_pay: Target total direct compensation column
    
    Constants:
    - positive_v: Value indicating positive eligibility
    - negative_v: Value indicating negative eligibility  
    - tenure_value: Tenure value for exclusion
    - fact_sti_threshold: Threshold for STI validation
    
    Returns:
    pd.DataFrame: Modified dataframe with calculated compensation elements
    """
    
    # Annual salary calculation
    df[annual_salary] = df[monthly_salary] / df[salary_rate] * df[number_annual_salaries]
    
    # Base pay calculation
    df[base_pay] = df[annual_salary] + np.where(df[additional_pay].isnull(), 0, df[additional_pay]) / df[salary_rate]
    
    # Actual STI output (with threshold filter)
    df[fact_sti_out] = np.where(
        (df[sti_eligibility] == negative_v) | (df[tenure] == tenure_value), 
        np.nan,
        np.where(
            (df[sti_eligibility] == positive_v) & 
            (df[fact_sti].notnull()) & 
            ((df[fact_sti] / df[base_pay]) > fact_sti_threshold), 
            df[fact_sti], 
            np.nan
        )
    )
    
    # Actual STI output alternative (without threshold filter)
    df[fact_sti_out_alt] = np.where(
        (df[sti_eligibility] == negative_v) | (df[tenure] == tenure_value), 
        np.nan,
        np.where(
            (df[sti_eligibility] == positive_v) & (df[fact_sti].notnull()), 
            df[fact_sti], 
            np.nan
        )
    )
    
    # Target STI output
    df[target_sti_out] = np.where(
        (df[sti_eligibility] == positive_v) & 
        (df[target_sti].notnull()) & 
        (df[target_sti] != 0), 
        df[target_sti], 
        np.nan
    )
    
    # Total cash pay
    df[tc_pay] = np.where(
        df[sti_eligibility] == negative_v, 
        df[base_pay],
        np.where(
            (df[sti_eligibility] == positive_v) & 
            (df[fact_sti].notnull()) & 
            (df[tenure] != tenure_value), 
            df[base_pay] + df[fact_sti_out_alt], 
            np.nan
        )
    )
    
    # LTIP pay (with threshold filter)
    df[ltip_pay] = np.where(
        (df[lti_eligibility] == negative_v) | (df[tenure] == tenure_value), 
        np.nan,
        np.where(
            (df[lti_eligibility] == positive_v) & 
            (df[fact_lti].notnull()) & 
            ((df[fact_lti] / df[base_pay]) > fact_sti_threshold), 
            df[fact_lti], 
            np.nan
        )
    )
    
    # LTIP pay alternative (without threshold filter)
    df[ltip_pay_alt] = np.where(
        (df[lti_eligibility] == negative_v) | (df[tenure] == tenure_value), 
        np.nan,
        np.where(
            (df[lti_eligibility] == positive_v) & (df[fact_lti].notnull()), 
            df[fact_lti], 
            np.nan
        )
    )
    
    # Target total cash pay
    df[ttc_pay] = np.where(
        df[sti_eligibility] == negative_v, 
        df[base_pay],
        np.where(
            (df[sti_eligibility] == positive_v) & 
            (df[target_sti].notnull()) & 
            (df[target_sti] != 0), 
            df[base_pay] * (1 + df[target_sti]), 
            np.nan
        )
    )
    
    # Target LTIP pay
    df[tltip_pay] = np.where(
        (df[lti_eligibility] == positive_v) & 
        (df[target_lti_per].notnull()) & 
        (df[target_lti_per] != 0), 
        df[target_lti_per], 
        np.nan
    )
    
    # Total direct compensation
    df[tdc_pay] = np.where(
        (df[sti_eligibility] == positive_v) & (df[fact_sti].isna()) |
        (df[lti_eligibility] == positive_v) & (df[fact_lti].isna()) |
        (df[tenure] == tenure_value) & ((df[sti_eligibility] == positive_v) | (df[lti_eligibility] == positive_v)),
        np.nan,
        np.where(
            (df[sti_eligibility].isna()) | (df[lti_eligibility].isna()),
            np.nan,
            df[base_pay] +
            np.where(
                (df[sti_eligibility] == negative_v) | (df[tenure] == tenure_value),
                0,
                np.where((df[sti_eligibility] == positive_v) & df[fact_sti].isna(), 0, df[fact_sti])
            ) +
            np.where(
                (df[lti_eligibility] == negative_v) | (df[tenure] == tenure_value),
                0,
                np.where((df[lti_eligibility] == positive_v) & df[fact_lti].isna(), 0, df[fact_lti])
            )
        )
    )
    
    # Target total direct compensation
    df[ttdc_pay] = np.where(
        (df[sti_eligibility] == positive_v) & (df[target_sti].isna() | (df[target_sti] == 0)) |
        (df[lti_eligibility] == positive_v) & (df[target_lti_per].isna() | (df[target_lti_per] == 0)),
        np.nan,
        df[base_pay] +
        np.where(df[sti_eligibility] == positive_v, df[base_pay] * df[target_sti], 0) +
        np.where(df[lti_eligibility] == positive_v, df[base_pay] * df[target_lti_per], 0)
    )
    
    # Clean up temporary columns
    df.drop([fact_sti_out_alt, ltip_pay_alt], axis=1, inplace=True)
    
    return df


ultimate_df = calculate_compensation_elements(
    ultimate_df,
    # Input columns
    monthly_salary=monthly_salary,
    salary_rate=salary_rate, 
    number_annual_salaries=number_monthly_salaries,
    additional_pay=additional_pay,
    sti_eligibility=sti_eligibility,
    tenure=tenure,
    fact_sti=fact_sti,
    target_sti=target_sti,
    fact_lti=fact_lti,
    target_lti_per=target_lti_per,
    lti_eligibility=lti_eligibility,
    
    # Output columns  
    annual_salary=annual_salary,
    base_pay=base_pay,
    fact_sti_out=fact_sti_out,
    fact_sti_out_alt=fact_sti_out_alt,
    target_sti_out=target_sti_out,
    tc_pay=tc_pay,
    ltip_pay=ltip_pay, 
    ltip_pay_alt=ltip_pay_alt,
    ttc_pay=ttc_pay,
    tltip_pay=tltip_pay,
    tdc_pay=tdc_pay,
    ttdc_pay=ttdc_pay,
    
    # Constants
    positive_v=positive_v,
    negative_v=negative_v,
    tenure_value=tenure_value, 
    fact_sti_threshold=0.05
)

In [23]:
def fact_lti_to_tc(df, lti_col, tc_col, output_col):
    """
    Compare fact LTI to TC values.
    Returns True if LTI < TC, or if either value is NaN.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    lti_col (str): Column name for LTI values
    tc_col (str): Column name for TC values  
    output_col (str): Output column name
    
    Returns:
    pd.DataFrame: Modified dataframe with comparison results
    """
    # Fixed: Compare df[lti_col] < df[tc_col], not df[lti_col] < tc_col
    # Fixed: tc_ol -> tc_col typo
    df[output_col] = (df[lti_col] < df[tc_col]) | (df[tc_col].isna()) | (df[lti_col].isna())
    return df

def target_lti_to_ttc(df, lti_col, ttc_col, base_pay_col, output_col):
    """
    Compare target LTI to TTC values.
    Returns True if (LTI * base_pay) < TTC, or if either value is NaN.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    lti_col (str): Column name for LTI values
    ttc_col (str): Column name for TTC values
    base_pay_col (str): Column name for base pay values
    output_col (str): Output column name
    
    Returns:
    pd.DataFrame: Modified dataframe with comparison results
    """
    # Fixed: Use df[ttc_col] instead of ttc_pay variable
    # Fixed: ttc_ol -> ttc_col typo
    # Fixed: Add base_pay_col parameter instead of hardcoded base_pay
    df[output_col] = ((df[lti_col] * df[base_pay_col]) < df[ttc_col]) | (df[ttc_col].isna()) | (df[lti_col].isna())
    return df

# Fixed version 1: Fact LTI to TC comparison
ultimate_df = fact_lti_to_tc(
    ultimate_df, 
    lti_col=fact_lti,      
    tc_col=tc_pay,         
    output_col='Fact LTI < TC'
)

# Fixed version 2: Target LTI to TTC comparison
ultimate_df = target_lti_to_ttc(
    ultimate_df,
    lti_col=tltip_pay,  
    ttc_col=ttc_pay,               
    base_pay_col=base_pay,         
    output_col='Target LTI < TTC'
)

In [24]:
def lower_mrot(df, input_col, output_col, mrot_value=268_800):
    """
    Compare column values against MROT threshold.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    input_col (str): Column name to compare against MROT
    output_col (str): Output column name for results
    mrot_value (float): MROT threshold value (default: 268,800)
    
    Returns:
    pd.DataFrame: DataFrame with new comparison column
    """
    df[output_col] = (df[input_col] > mrot_value) | (df[input_col].isna())
    return df

# Assuming your columns are named 'tc_pay' and 'ttc_pay'
ultimate_df = lower_mrot(ultimate_df, tc_pay, 'TC > MROT')
ultimate_df = lower_mrot(ultimate_df, ttc_pay, 'TTC > MROT') 

# Get rows where TC OR TTC is below MROT
low_tc_mask = ultimate_df['TC > MROT'] == False
low_ttc_mask = ultimate_df['TTC > MROT'] == False
lower_mrot_df = ultimate_df[low_tc_mask | low_ttc_mask].copy()

In [25]:
def ti_higher(df, input_col, output_col, threshold=3):
    df[output_col] = (df[input_col] < threshold) | (df[input_col].isna())
    return df

ultimate_df = ti_higher(ultimate_df, target_sti_out, 'Normal TI', 3)
high_ti = ultimate_df[ultimate_df['Normal TI'] == False]

In [26]:
def lti_checks(df, main_lti, lti_1, lti_2, lti_3, output_col):
    df[output_col] = (df[main_lti] == (df[lti_1] + df[lti_2] + df[lti_3])) | df[main_lti].isna()
    return df

ultimate_df = lti_checks(ultimate_df, fact_lti, fact_lti_1, fact_lti_2, fact_lti_3, 'Fact LTI = Fact LTI Parts')
ultimate_df = lti_checks(ultimate_df, target_lti_per, target_lti_1, target_lti_2, target_lti_3, 'Target LTI = Target LTI Parts')

In [27]:
# Function to compare how realistic is the certain compensation element
def validate_compensation_ranges(df, comp_column, grade_col, output_col, comp_type='BP'):
    
    # Get input data
    compensation_values = df[comp_column].copy()
    grades = pd.to_numeric(df[grade_col], errors='coerce')
    
    # Initialize result with False
    result = pd.Series(False, index=df.index, dtype=bool)
    
    # Handle NaN compensation values - set to True
    nan_comp_mask = compensation_values.isna()
    result.loc[nan_comp_mask] = True
    
    # Create lookup series for min and max values
    grade_min_map = {}
    grade_max_map = {}
    
    for grade, comp_data in crazy_ranges.items():
        if comp_type in comp_data:
            grade_min_map[grade] = comp_data[comp_type]['min']
            grade_max_map[grade] = comp_data[comp_type]['max']
    
    # Convert to pandas Series for vectorized operations
    min_values = grades.map(grade_min_map)
    max_values = grades.map(grade_max_map)
    
    # Vectorized comparison: compensation is within range
    valid_grade_mask = grades.isin(crazy_ranges.keys())
    within_range_mask = (
        valid_grade_mask & 
        ~nan_comp_mask & 
        (compensation_values >= min_values) & 
        (compensation_values <= max_values)
    )
    
    # Set True for values within range
    result.loc[within_range_mask] = True
    
    # Create output column
    df[output_col] = result
    
    return df

In [28]:
ultimate_df = validate_compensation_ranges(
    ultimate_df, 
    comp_column=base_pay,
    grade_col=grade, 
    output_col='BP_within_range',
    comp_type='BP'
)

ultimate_df = validate_compensation_ranges(
    ultimate_df, 
    comp_column=tc_pay,
    grade_col=grade, 
    output_col='TC_within_range',
    comp_type='TC'
)

ultimate_df = validate_compensation_ranges(
    ultimate_df, 
    comp_column=ttc_pay,
    grade_col=grade, 
    output_col='TTC_within_range',
    comp_type='TTC'
)


# Make sure to delete whitespaces from
ultimate_df[gi_origin] = ultimate_df[gi_origin].where(
    ultimate_df[gi_origin].isna(),  # keep NaNs as-is
    ultimate_df[gi_origin].astype(str).str.strip()
)

In [29]:
print(f'Size of the df: {ultimate_df.shape[0]}')

check_end = time.time()
print(f'Checks completed in: {check_end - check_start}')

Size of the df: 2464
Checks completed in: 2.5949854850769043


In [30]:
print(ultimate_df[gi_sector].unique())

['Розничная торговля' 'ИТ']


In [31]:
# def export_by_sectors(ultimate_df, lower_mrot_df, high_ti, sector_column, output_folder):
#     """
#     Simple function to export data split by 4 employee groups.
#     """
    
#     # Define your 4 groups and their sectors here
#     groups = {
#         'Olga': ['Sector1', 'Sector2', 'Sector3'],  # Replace with your actual sector names
#         'Anna': ['Sector4', 'Sector5', 'Sector6'],  # Replace with your actual sector names  
#         'Danila': ['Sector7', 'Sector8', 'Sector9'],  # Replace with your actual sector names
#         'Ilya': ['Sector10', 'Sector11']            # Replace with your actual sector names
#     }
    
#     # Loop through each group
#     for group_name, sectors in groups.items():
        
#         # Filter data for this group's sectors
#         group_mask = ultimate_df[sector_column].isin(sectors)
#         group_ultimate_df = ultimate_df[group_mask]
        
#         # Filter other dataframes if they have the sector column
#         if sector_column in lower_mrot_df.columns:
#             group_lower_mrot = lower_mrot_df[lower_mrot_df[sector_column].isin(sectors)]
#         else:
#             group_lower_mrot = lower_mrot_df  # Use all data if no sector column
            
#         if sector_column in high_ti.columns:
#             group_high_ti = high_ti[high_ti[sector_column].isin(sectors)]
#         else:
#             group_high_ti = high_ti  # Use all data if no sector column
        
#         # Create filename
#         filename = f'Database_{group_name}.xlsx'
#         file_path = os.path.join(output_folder, filename)
        
#         # Export to Excel
#         with pd.ExcelWriter(file_path) as writer:
#             group_ultimate_df.to_excel(writer, index=False, sheet_name='Total Data')
#             group_lower_mrot.to_excel(writer, index=True, sheet_name='Lower than MROT')
#             group_high_ti.to_excel(writer, index=True, sheet_name='High TI')
        
# #         print(f"Exported {len(group_ultimate_df)} records for {group_name} to {filename}")

# export_by_sectors(
#     ultimate_df=ultimate_df,
#     lower_mrot_df=lower_mrot_df,
#     high_ti=high_ti,
#     sector_column=gi_sector,
#     output_folder=output_folder
# )

In [32]:
try:
    output_path = os.path.join(output_folder, 'Database.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        ultimate_df.to_excel(writer, index=False, sheet_name='Total Data')
        lower_mrot_df.to_excel(writer, index=True, sheet_name='Lower than MROT')
        high_ti.to_excel(writer, index=True, sheet_name='High TI')
    print(f"Successfully saved Excel file to: {output_path}")
except Exception as e:
    print(f"Failed to save Excel file: {e}")
    # Code continues here regardless of the error

Successfully saved Excel file to: output\Database.xlsx


### Checking if there are still FALSE left in the df

- Поиск строк, где хотя бы одна проверка не прошла (False).
- false_df это финальный DataFrame со всеми ошибочными кейсами. Экспорт в false_records.xlsx.

In [33]:
# Function to filter
def filter_any_false(df, boolean_columns):
    """
    Filter DataFrame to keep only rows where ANY of the specified boolean columns is False
    
    Parameters:
    df: pandas DataFrame
    boolean_columns: list of column names (should contain boolean values)
    
    Returns:
    DataFrame with rows where at least one specified column is False
    """
    # Create a mask where at least one of the specified columns is False
    mask = ~df[boolean_columns].all(axis=1)
    
    return df[mask]

In [34]:
# boolean_columns = ['Grade_Check','Fun Code Validity','Sub Code Validity','Spec Code Validity',
#                    'Region Validity','Sub in Fun','Spec in Sub','Ставка','Число окладов','Оклад указан',
#                    'STI_eligibility','LTI_eligibility', 'Tenure Value', 'Code_Grade_Val', 'LTI_Grade_Val',
#                    'Fact LTI < TC', 'Target LTI < TTC', 'TC > MROT', 'TTC > MROT', 'Normal TI',
#                    'Fact LTI = Fact LTI Parts', 'Target LTI = Target LTI Parts', 'BP_within_range',
#                    'TC_within_range', 'TTC_within_range']

boolean_cols = ultimate_df.select_dtypes(include=['bool']).columns.tolist()
print(boolean_cols)

['Grade_Check', 'Fun Code Validity', 'Sub Code Validity', 'Spec Code Validity', 'Region Validity', 'Sub in Fun', 'Spec in Sub', 'Ставка', 'Число окладов', 'Оклад указан', 'STI_eligibility', 'LTI_eligibility', 'Tenure Value', 'Fact LTI < TC', 'Target LTI < TTC', 'TC > MROT', 'TTC > MROT', 'Normal TI', 'Fact LTI = Fact LTI Parts', 'Target LTI = Target LTI Parts', 'BP_within_range', 'TC_within_range', 'TTC_within_range']


In [35]:
false_df = filter_any_false(ultimate_df, boolean_cols)

In [36]:
# Check if false_df is empty and export accordingly
if false_df.empty:
    print("DataFrame is empty - nothing was exported")
else:
    false_df.to_excel('false_records.xlsx', index=False)
    print(f"DataFrame exported to 'false_records.xlsx' with {len(false_df)} rows")

DataFrame exported to 'false_records.xlsx' with 19 rows


In [37]:
false_df

,Название компании (заполняется автоматически),Подразделение 1 уровня,Подразделение 2 уровня,Подразделение 3 уровня,Подразделение 4 уровня,Подразделение 5 уровня,Подразделение 6 уровня,Название должности,Код сотрудника,Код руководителя сотрудника,...,Fact LTI < TC,Target LTI < TTC,TC > MROT,TTC > MROT,Normal TI,Fact LTI = Fact LTI Parts,Target LTI = Target LTI Parts,BP_within_range,TC_within_range,TTC_within_range
280,12_STOREEZ,"ООО ""12 историй""",Управление маркетинга,Блок продакшена,Служба рекламного продакшена,NaN,NaN,Продакшн директор,ИСЗК-04190,NaN,...,True,True,True,True,True,True,True,True,True,False
359,12_STOREEZ,"ООО ""12 историй""",Управление розничных продаж (Дивизион 1),NaN,NaN,NaN,NaN,Продавец-консультант 1 категории,ИСЗК-03082,NaN,...,True,True,True,True,True,True,True,True,True,False
434,12_STOREEZ,"ООО ""12 историй""",Управление розничных продаж (Дивизион 1),NaN,NaN,NaN,NaN,Продавец-консультант 1 категории,ИСЗК-02266,NaN,...,True,True,True,True,True,True,True,True,False,True
35,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок серверного оборудования,Менеджер по развитию серверного оборудования,338,NaN,...,True,True,True,True,False,True,True,True,True,True
48,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок компьютерных комплектующих,Старший менеджер по закупкам,118,NaN,...,True,True,True,True,False,True,True,True,True,False
274,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок компьютерных комплектующих,Старший менеджер по закупкам,1025,NaN,...,True,True,True,True,False,True,True,True,True,False
391,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок серверного оборудования,Менеджер по закупкам,502,NaN,...,True,True,True,True,False,True,True,True,True,True
532,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок готовых решений,Руководитель отдела,517,NaN,...,True,True,True,True,False,True,True,True,True,True
675,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок компьютерных комплектующих,Старший менеджер по закупкам,9,NaN,...,True,True,True,True,False,True,True,True,True,False
732,3Logic_Group,Новый Ай Ти Проект ООО,Коммерческий блок,Коммерческий блок,Коммерческий блок,Департамент закупок,Отдел закупок компьютерных комплектующих,Старший менеджер по закупкам,177,NaN,...,True,True,True,True,False,True,True,True,True,False


### Creating parqeut file for market data calculation

- Приведение к нужным типам (str, float, int).
- Удаление nan в строках.
- Сохранение в parquet. 

In [38]:
final_df = ultimate_df.copy()
final_df = final_df[expected_columns_market_df]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2464 entries, 0 to 1419
Data columns (total 66 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Название компании (заполняется автоматически)                                     2464 non-null   object 
 1   Подразделение 1 уровня                                                            2464 non-null   object 
 2   Подразделение 2 уровня                                                            2464 non-null   object 
 3   Подразделение 3 уровня                                                            1904 non-null   object 
 4   Подразделение 4 уровня                                                            1603 non-null   object 
 5   Подразделение 5 уровня                                                            1420 non-null   object 
 6   Подра

In [39]:
# Setting the datatype for sting columns
string_columns = [company_name, dep_level_1, dep_level_2, dep_level_3, dep_level_4, dep_level_5, dep_level_6,
                  job_title, employee_code, manager_code, man_emp, performance, n_level, expat, gender_id, bod,
                  hired_date, tenure, region_client_fill, region, internal_grade, grade, function_code, subfunction_code,
                  specialization_code, function, subfunction, specialization, lti_eligibility, lti_prog_1, 
                  lti_pay_freq_1, lti_prog_2, lti_pay_freq_2, lti_prog_3, lti_pay_freq_3, comments, macroregion, 
                  gi_sector, gi_origin, gi_headcount_cat, gi_revenue_cat, sti_eligibility]

# Setting sting types
for col in string_columns:
    final_df[col] = final_df[col].astype(str)
    final_df[col] = final_df[col].str.strip()

# Removing nan after thew data was stringged
for col in string_columns:
    final_df[col] = final_df[col].replace('nan', np.nan)

# Replacing np.nan in specialization with '-'
final_df[specialization] = final_df[specialization].replace(np.nan, '-')

In [40]:
# Setting the datatype for float columns
float_columns = [salary_rate, monthly_salary, number_monthly_salaries, additional_pay, 
                 fact_sti, target_sti, fact_lti, target_lti_per, target_lti_1, 
                 target_lti_2, target_lti_3, fact_lti_1,fact_lti_2, fact_lti_3, annual_salary, base_pay, 
                 fact_sti_out, target_sti_out, tc_pay, ttc_pay, ltip_pay, tltip_pay, tdc_pay, ttdc_pay]

for col in float_columns:
    # Remove non-numeric values by coercing errors
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')
    final_df[col] = final_df[col].astype(float)

In [41]:
# Setting the datatype for int columns
int_columns = [grade]

for col in int_columns:
    # Remove non-numeric values by coercing errors
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

    # Find all non-finite values (NaN, inf, -inf)
    non_finite_values = final_df[col][~np.isfinite(final_df[col])]  # ~np.isfinite() selects non-finite values
    # non_finite_values.to_excel("NON-finite.xlsx")
    print("Non-finite values causing issues:")
    print(non_finite_values)

    final_df[col] = final_df[col].astype('Int64')   # лучше чем int, потому что поддерживает NaN


Non-finite values causing issues:
Series([], Name: Грейд / Уровень обзора, dtype: float64)


In [42]:
final_df.to_parquet('Rawdata_2025_before_reload.parquet.gzip', compression='gzip')

In [43]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2464 entries, 0 to 1419
Data columns (total 66 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Название компании (заполняется автоматически)                                     2464 non-null   object 
 1   Подразделение 1 уровня                                                            2464 non-null   object 
 2   Подразделение 2 уровня                                                            2464 non-null   object 
 3   Подразделение 3 уровня                                                            1904 non-null   object 
 4   Подразделение 4 уровня                                                            1603 non-null   object 
 5   Подразделение 5 уровня                                                            1420 non-null   object 
 6   Подра

In [44]:
end_time = time.time()
print(f'Total time: {end_time - start_time}')

Total time: 31.420121669769287


In [45]:
res = pd.read_parquet('Rawdata_2025_before_reload.parquet.gzip')
res

,Название компании (заполняется автоматически),Подразделение 1 уровня,Подразделение 2 уровня,Подразделение 3 уровня,Подразделение 4 уровня,Подразделение 5 уровня,Подразделение 6 уровня,Название должности,Код сотрудника,Код руководителя сотрудника,...,Годовой оклад (AP),Базовый оклад (BP),Краткосрочное фактическое переменное вознаграждение (VP),Целевая Премия (TI),Фактическое совокупное вознаграждение (TC),Целевое совокупное вознаграждение (TTC),Фактическое долгосрочное вознаграждение (LTIP),Целевое долгосрочное вознаграждение (TLTIP),Прямое совокупное вознаграждение (TDC),Целевое прямое совокупное вознаграждение (TTDC)
0,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,None,None,Инженер технической поддержки 1С,ИТЗК-00105,None,...,1620000.00,1620000.00,243000.0,0.15000,1863000.0,1.863000e+06,NaN,NaN,1863000.0,1.863000e+06
1,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,None,None,Инженер технической поддержки приложений,ИТЗК-00250,None,...,1648002.84,1648002.84,NaN,0.14745,NaN,1.891001e+06,NaN,NaN,NaN,1.891001e+06
2,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,None,None,Инженер технической поддержки офиса,ИТЗК-00176,None,...,1793100.00,1793100.00,268964.0,0.15000,2062064.0,2.062065e+06,NaN,NaN,2062064.0,2.062065e+06
3,12_STOREEZ,АйТи Фаундейшн ООО,Финансово-стратегическое управление,Блок корпоративных финансов,None,None,None,Бухгалтер-казначей,ИТЗК-00139,None,...,1440000.00,1440000.00,105600.0,0.07333,1545600.0,1.545595e+06,NaN,NaN,1545600.0,1.545595e+06
4,12_STOREEZ,АйТи Фаундейшн ООО,Управление разработки и производства продукта,None,None,None,None,Ведущий методолог анализа Продукта,ИТЗК-00167,None,...,4140000.00,4140000.00,450000.0,0.10870,4590000.0,4.590018e+06,NaN,NaN,4590000.0,4.590018e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,3Logic_Group,Новый Ай Ти Проект ООО,"Обособленное подразделение ""Холмогоры""","Обособленное подразделение ""Холмогоры""","Обособленное подразделение ""Холмогоры""",Складской комплекс,Направление электронной коммерции,Кладовщик 1 категории,2119,None,...,900276.60,900276.60,NaN,NaN,900276.6,9.002766e+05,NaN,NaN,900276.6,9.002766e+05
1416,3Logic_Group,Новый Ай Ти Проект ООО,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Отдел проектно-продуктового управления,Менеджер проектов,1819,None,...,3704982.00,3704982.00,NaN,0.24900,NaN,4.627523e+06,NaN,NaN,NaN,4.627523e+06
1417,3Logic_Group,Новый Ай Ти Проект ООО,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Отдел разработки программного обеспечения,Группа тестирования ОРПО,Руководитель группы,576,None,...,3447180.00,3447180.00,NaN,0.33187,NaN,4.591196e+06,NaN,NaN,NaN,4.591196e+06
1418,3Logic_Group,Новый Ай Ти Проект ООО,Технический блок,Технический блок,Технический блок,Технический блок,Департамент исследований и разработки,Младший инженер-программист,РТЗК-00064,None,...,925531.20,925531.20,NaN,0.65575,NaN,1.532448e+06,NaN,NaN,NaN,1.532448e+06


In [46]:
res_db = pd.read_excel('output/Database.xlsx')

In [47]:
res_db

,Название компании (заполняется автоматически),Подразделение 1 уровня,Подразделение 2 уровня,Подразделение 3 уровня,Подразделение 4 уровня,Подразделение 5 уровня,Подразделение 6 уровня,Название должности,Код сотрудника,Код руководителя сотрудника,...,Fact LTI < TC,Target LTI < TTC,TC > MROT,TTC > MROT,Normal TI,Fact LTI = Fact LTI Parts,Target LTI = Target LTI Parts,BP_within_range,TC_within_range,TTC_within_range
0,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,NaN,NaN,Инженер технической поддержки 1С,ИТЗК-00105,NaN,...,True,True,True,True,True,True,True,True,True,True
1,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,NaN,NaN,Инженер технической поддержки приложений,ИТЗК-00250,NaN,...,True,True,True,True,True,True,True,True,True,True
2,12_STOREEZ,АйТи Фаундейшн ООО,Управление информационных технологий,Блок ИТ-инфраструктуры и сервиса,Служба технической поддержки,NaN,NaN,Инженер технической поддержки офиса,ИТЗК-00176,NaN,...,True,True,True,True,True,True,True,True,True,True
3,12_STOREEZ,АйТи Фаундейшн ООО,Финансово-стратегическое управление,Блок корпоративных финансов,NaN,NaN,NaN,Бухгалтер-казначей,ИТЗК-00139,NaN,...,True,True,True,True,True,True,True,True,True,True
4,12_STOREEZ,АйТи Фаундейшн ООО,Управление разработки и производства продукта,NaN,NaN,NaN,NaN,Ведущий методолог анализа Продукта,ИТЗК-00167,NaN,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,3Logic_Group,Новый Ай Ти Проект ООО,"Обособленное подразделение ""Холмогоры""","Обособленное подразделение ""Холмогоры""","Обособленное подразделение ""Холмогоры""",Складской комплекс,Направление электронной коммерции,Кладовщик 1 категории,2119,NaN,...,True,True,True,True,True,True,True,True,True,True
2460,3Logic_Group,Новый Ай Ти Проект ООО,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Отдел проектно-продуктового управления,Менеджер проектов,1819,NaN,...,True,True,True,True,True,True,True,True,True,True
2461,3Logic_Group,Новый Ай Ти Проект ООО,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Департамент информационных технологий и цифров...,Отдел разработки программного обеспечения,Группа тестирования ОРПО,Руководитель группы,576,NaN,...,True,True,True,True,True,True,True,True,True,True
2462,3Logic_Group,Новый Ай Ти Проект ООО,Технический блок,Технический блок,Технический блок,Технический блок,Департамент исследований и разработки,Младший инженер-программист,РТЗК-00064,NaN,...,True,True,True,True,True,True,True,True,True,True
